<a href="https://colab.research.google.com/github/subchina/LLM-Project/blob/main/PEFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1t30Elo92Ti8F3BOorJtVGF4MQfjMskk-


Downloading...
From: https://drive.google.com/uc?id=1t30Elo92Ti8F3BOorJtVGF4MQfjMskk-
To: /content/cwe_prompt_completion.json
100% 16.6M/16.6M [00:00<00:00, 113MB/s] 


In [2]:
!pip install transformers accelerate peft bitsandbytes datasets


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
import torch
from peft import LoraConfig, get_peft_model
import bitsandbytes as bnb

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig,
    DataCollatorForLanguageModeling
)

# Load the Mistral 7B model and tokenizer

compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

device_map = {"": 0}
tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-v0.1',trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    'mistralai/Mistral-7B-v0.1',
    device_map=device_map,
    quantization_config=bnb_config,
    trust_remote_code=True,
    use_auth_token=True)

# Set up QLoRA configurations
lora_config = LoraConfig(
    r=64,  # Rank of the LoRA (this value can be tuned)
    lora_alpha=16,  # Scaling factor
    target_modules=['q_proj', 'v_proj', 'k_proj', 'dense'],  # Apply LoRA to specific layers
    lora_dropout=0.05,  # Dropout rate for regularization
    bias="none",  # No bias term in LoRA
    task_type="CAUSAL_LM"  # Type of task for language modeling
)



# Apply LoRA to the quantized model
model = get_peft_model(model, lora_config)

model.gradient_checkpointing_enable()



tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [5]:
from datasets import load_dataset
import pyarrow as pa

# Load the dataset
data_path = "cwe_prompt_completion.json"
dataset = load_dataset('json', data_files=data_path)



Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('prompt', 'completion').
    Then concatenate them using two newline characters.

    :param sample: Sample dictionary containing 'prompt' and 'completion'.
    """
    INTRO_BLURB = "Below is a question related to a security vulnerability. Provide a detailed and accurate response."
    PROMPT_KEY = "### Prompt:"
    RESPONSE_KEY = "### Response:"
    END_KEY = "### End"

    blurb = f"\n{INTRO_BLURB}"
    prompt = f"{PROMPT_KEY}\n{sample['prompt']}" if sample["prompt"] else None
    response = f"{RESPONSE_KEY}\n{sample['completion']}" if sample["completion"] else None
    end = f"{END_KEY}"

    parts = [part for part in [blurb, prompt, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt

    return sample


In [7]:
from functools import partial

# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int,seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """

    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)#, batched=True)

    # Apply preprocessing to each batch of the dataset & and remove 'instruction', 'context', 'response', 'category' fields
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True
    )

    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset

In [8]:
## Pre-process dataset
from transformers import set_seed
seed = 42
set_seed(seed)
max_length = get_max_length(model)
print(max_length)

#TODO reverted to orignal code - removal of columns, there is no validation set in the dataset, you can potentially fix it or ignore it completely
train_dataset = preprocess_dataset(tokenizer, max_length,seed, dataset['train'])
#eval_dataset = preprocess_dataset(tokenizer, max_length,seed, dataset['validation'])

Found max lenth: 32768
32768
Preprocessing dataset...


Map:   0%|          | 0/32654 [00:00<?, ? examples/s]

Map:   0%|          | 0/32654 [00:00<?, ? examples/s]

Filter:   0%|          | 0/32654 [00:00<?, ? examples/s]

In [9]:
from peft import get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)


/usr/local/lib/python3.10/dist-packages/peft/mapping.py:172: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'mistralai/Mistral-7B-v0.1' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(


In [10]:
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)


# Tokenize the dataset
tokenized_datasets = train_dataset.map(tokenize_function, batched=True)

# Prepare the dataset for PyTorch
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask"])

Map:   0%|          | 0/32654 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments

peft_training_args = TrainingArguments(
    output_dir = './mistral-finetune',
    warmup_steps=1,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    max_steps=1000,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    evaluation_strategy="steps",
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
)

model.config.use_cache = False
tokenizer.pad_token = tokenizer.eos_token

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=peft_training_args,
    train_dataset=tokenized_datasets,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)

)

# Start fine-tuning the model
trainer.train()
# Save the fine-tuned model
model.save_pretrained("./mistral-qlora-finetune")
tokenizer.save_pretrained("./mistral-qlora-finetune")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_